# Optimización de una regresión lineal.

Generación datos aleatoriamente a partir de un modelo de regresión lineal predefinido donde $-5 \leq \beta_{i} \leq 5 \quad i = 1, \dots , K$ con al menos $K = 100$ variables explicativas y un mínimo de $n = 1000$ observaciones. Queremos ajustar un modelo de regresión lineal múltiple que explique la variable $Y$ en función de las demás $X (Y = \beta^T X+\epsilon)$, donde $\beta = (\beta_{0}, \beta_{1}, \beta_{2}, \dots, \beta_{k})^T$, usando "*Least Squares Estimation*":

\begin{align*}
\underset{\beta}{\min} & \quad \sum_{i=1}^{n} (y_{i}-\beta^Tx_{i})^2\\
\end{align*}

In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
np.random.seed(42)
nsample = 1000
nvariables=100
X0 = np.ones([nsample,1]) # columna Beta_0, luego se concatena con las betas_i
Xi = np.random.random([nsample,nvariables]) # Los valores de x_i están entre 0 y 1
X = np.concatenate([X0, Xi],axis=1)
beta = np.random.uniform(-5,5,size=([nvariables+1,1]))
epsilon=np.random.normal(0,1,(nsample,1)) # El error se distribuye según una normal
Y = np.dot(X,beta)+epsilon

### a) Estimación el valor de los parámetros de la regresión aplicando la solución analítica.

In [3]:
from numpy.linalg import inv
time_start = time.clock()
beta_ls_exact = np.dot(np.dot(inv(np.dot(X.T,X)),X.T),Y)
time_elapsed = (time.clock() - time_start)
print('time elapsed=',time_elapsed)
beta_ls_exact

time elapsed= 0.010654578036345183


array([[-0.15233413],
       [ 0.08546847],
       [-1.17631577],
       [-0.14108304],
       [-1.40110843],
       [ 1.45466706],
       [ 3.44572396],
       [ 2.91711795],
       [-1.61313217],
       [-0.1533509 ],
       [-5.2159183 ],
       [ 3.83021278],
       [-2.49119516],
       [-0.50809872],
       [-3.48931602],
       [ 1.93092301],
       [ 0.36183479],
       [-1.07269884],
       [ 0.43490732],
       [-3.30353155],
       [ 3.02848066],
       [-2.24520816],
       [ 1.09712707],
       [ 3.97565127],
       [-4.55225206],
       [-1.0815387 ],
       [-3.62442509],
       [ 3.86066222],
       [-1.30016657],
       [-2.93455736],
       [ 1.8001133 ],
       [ 0.38632804],
       [ 3.26473816],
       [ 2.59868212],
       [ 4.79013726],
       [-3.82204707],
       [-4.20181943],
       [ 4.29823584],
       [ 4.69180638],
       [-1.10880057],
       [-0.1686732 ],
       [-1.47431859],
       [ 2.66300588],
       [ 2.71941881],
       [-2.64873996],
       [-1

### b) Estimación el valor de los coeficientes de la regresión usando la herramienta minimize del paquete de Python scipy.optimize. Se prueban cuatro solvers diferentes y se comparan su eficiencia en términos de: número de iteraciones totales, número de evaluaciones de la función objetivo, gradiente y hesiano, así como el tiempo de computo total.

In [4]:
from scipy.optimize import minimize

Fución de regresión que utilizamos:

In [5]:
def least_sq_reg(beta_ls, X, Y):
    beta_ls = np.matrix(beta_ls)
    z = Y-np.dot(X,beta_ls.T)
    return np.dot(z.T,z)

Función del *jacobiano*, derivada 1ª

In [6]:
def least_sq_reg_jac(beta_ls,X,Y):
    beta_ls = np.matrix(beta_ls)
    pp = -2*np.dot((Y-np.dot(X,(beta_ls).T)).T,X)
    aa = np.squeeze(np.asarray(pp)) # NO ENTIENDO np.squeeze, ni para que hace un cast de array (np.asarray)
    return aa


Función del *hessiano*, derivada 2ª

In [7]:
def least_sq_reg_hess(beta_ls,X,Y):
    ss = 2*np.dot(np.transpose(X),X) # NO ENTIENDO el resultado de la derivada 2ª. A mi me sale solo 2·X, no 2·X^2
    return ss

Genramos un dataframe con los resultados

In [8]:
df_comparativo = pd.DataFrame(columns=['ejercicio','n_iter','n_eval','n_grad','n_hess','total_time_elapsed','iter_time_elapsed','error'])

#### Regresión lineal con el Solver *SLSQP*

In [9]:
beta_ls_ini = np.zeros(nvariables+1)
time_start = time.clock()
method = 'SLSQP'
res = minimize(least_sq_reg, beta_ls_ini, args=(X, Y), method=method, options={'disp': True,'xtol': 1e-10})
time_elapsed = (time.clock() - time_start)
error = np.linalg.norm(beta_ls_exact.T - res.x,ord=2)/np.linalg.norm(beta_ls_exact.T,ord=2)
ejercicio = '1b'
aux = [ejercicio,res.nit, res.nfev]
try:
    aux.append(res.njev)
except Exception as e:
    aux.append(None)
try:
    aux.append(res.nhev)
except Exception as e:
    aux.append(None)
aux = aux + ['{:.8f}'.format(time_elapsed), '{:.8f}'.format(time_elapsed/res.nit),error]
aux = pd.DataFrame(aux).T
aux.columns = ['ejercicio','n_iter','n_eval','n_grad','n_hess','total_time_elapsed','iter_time_elapsed','error']
aux.index = [method]
df_comparativo = pd.concat([df_comparativo,aux])

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: OptimizeWarning: Unknown solver options: xtol
  after removing the cwd from sys.path.


Optimization terminated successfully.    (Exit mode 0)
            Current function value: 953.7738898681563
            Iterations: 41
            Function evaluations: 4310
            Gradient evaluations: 41


#### Regresión lineal con el Solver *CG*

In [10]:
beta_ls_ini = np.zeros(nvariables+1)
time_start = time.clock()
method = 'CG'
res = minimize(least_sq_reg, beta_ls_ini, args=(X, Y), method=method, jac=least_sq_reg_jac, options={'disp': True,'xtol': 1e-10})
time_elapsed = (time.clock() - time_start)
error = np.linalg.norm(beta_ls_exact.T - res.x,ord=2)/np.linalg.norm(beta_ls_exact.T,ord=2)
ejercicio = '1b'
aux = [ejercicio,res.nit, res.nfev]
try:
    aux.append(res.njev)
except Exception as e:
    aux.append(None)
try:
    aux.append(res.nhev)
except Exception as e:
    aux.append(None)
aux = aux + ['{:.8f}'.format(time_elapsed), '{:.8f}'.format(time_elapsed/res.nit),error]
aux = pd.DataFrame(aux).T
aux.columns = ['ejercicio','n_iter','n_eval','n_grad','n_hess','total_time_elapsed','iter_time_elapsed','error']
aux.index = [method]
df_comparativo = pd.concat([df_comparativo,aux])

Optimization terminated successfully.
         Current function value: 953.773888
         Iterations: 67
         Function evaluations: 149
         Gradient evaluations: 149


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: OptimizeWarning: Unknown solver options: xtol
  after removing the cwd from sys.path.


#### Regresión lineal con el Solver *Quasi-Newton (BFGS)*

In [11]:
beta_ls_ini = np.zeros(nvariables+1)
time_start = time.clock()
method = 'BFGS'
res = minimize(least_sq_reg, beta_ls_ini, args=(X, Y), method=method, jac=least_sq_reg_jac, options={'disp': True,'xtol': 1e-10})
time_elapsed = (time.clock() - time_start)
error = np.linalg.norm(beta_ls_exact.T - res.x,ord=2)/np.linalg.norm(beta_ls_exact.T,ord=2)
ejercicio = '1b'
aux = [ejercicio,res.nit, res.nfev]
try:
    aux.append(res.njev)
except Exception as e:
    aux.append(None)
try:
    aux.append(res.nhev)
except Exception as e:
    aux.append(None)
aux = aux + ['{:.8f}'.format(time_elapsed), '{:.8f}'.format(time_elapsed/res.nit),error]
aux = pd.DataFrame(aux).T
aux.columns = ['ejercicio','n_iter','n_eval','n_grad','n_hess','total_time_elapsed','iter_time_elapsed','error']
aux.index = ['Quasi-Newton (BFGS)']
df_comparativo = pd.concat([df_comparativo,aux])

Optimization terminated successfully.
         Current function value: 953.773888
         Iterations: 133
         Function evaluations: 191
         Gradient evaluations: 191


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: OptimizeWarning: Unknown solver options: xtol
  after removing the cwd from sys.path.


#### Regresión lineal con el Solver *Trust-exact*

In [12]:
beta_ls_ini = np.zeros(nvariables+1)
time_start = time.clock()
method = 'trust-exact'
res = minimize(least_sq_reg, beta_ls_ini, args=(X, Y), method= method, jac=least_sq_reg_jac, hess=least_sq_reg_hess, options={'disp': True,'xtol': 1e-10})
time_elapsed = (time.clock() - time_start)
error = np.linalg.norm(beta_ls_exact.T - res.x,ord=2)/np.linalg.norm(beta_ls_exact.T,ord=2)
ejercicio = '1b'
aux = [ejercicio,res.nit, res.nfev]
try:
    aux.append(res.njev)
except Exception as e:
    aux.append(None)
try:
    aux.append(res.nhev)
except Exception as e:
    aux.append(None)
aux = aux + ['{:.8f}'.format(time_elapsed), '{:.8f}'.format(time_elapsed/res.nit),error]
aux = pd.DataFrame(aux).T
aux.columns = ['ejercicio','n_iter','n_eval','n_grad','n_hess','total_time_elapsed','iter_time_elapsed','error']
aux.index = [method]
df_comparativo = pd.concat([df_comparativo,aux])

Optimization terminated successfully.
         Current function value: 953.773888
         Iterations: 5
         Function evaluations: 6
         Gradient evaluations: 6
         Hessian evaluations: 6


C:\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:627: OptimizeWarning: Unknown solver options: xtol
  callback=callback, **options)


#### Regresión lineal con el Solver *Newton-CG*

In [13]:
beta_ls_ini = np.zeros(nvariables+1)
time_start = time.clock()
method = 'Newton-CG'
res = minimize(least_sq_reg, beta_ls_ini, args=(X, Y), method=method, jac=least_sq_reg_jac, hess=least_sq_reg_hess, options={'disp': True,'xtol': 1e-10})
time_elapsed = (time.clock() - time_start)
error = np.linalg.norm(beta_ls_exact.T - res.x,ord=2)/np.linalg.norm(beta_ls_exact.T,ord=2)
ejercicio = '1b'
aux = [ejercicio,res.nit, res.nfev]
try:
    aux.append(res.njev)
except Exception as e:
    aux.append(None)
try:
    aux.append(res.nhev)
except Exception as e:
    aux.append(None)
aux = aux + ['{:.8f}'.format(time_elapsed), '{:.8f}'.format(time_elapsed/res.nit),error]
aux = pd.DataFrame(aux).T
aux.columns = ['ejercicio','n_iter','n_eval','n_grad','n_hess','total_time_elapsed','iter_time_elapsed','error']
aux.index = [method]
df_comparativo = pd.concat([df_comparativo,aux])

Optimization terminated successfully.
         Current function value: 953.773888
         Iterations: 17
         Function evaluations: 57
         Gradient evaluations: 69
         Hessian evaluations: 17


In [14]:
df_comparativo

,ejercicio,n_iter,n_eval,n_grad,n_hess,total_time_elapsed,iter_time_elapsed,error
SLSQP,1b,41,4310,41,None,0.30910774,0.00753921,5.50764e-06
CG,1b,67,149,149,None,0.04317675,0.00064443,7.33548e-09
Quasi-Newton (BFGS),1b,133,191,191,None,0.09339162,0.00070219,3.05078e-14
trust-exact,1b,5,6,6,6,0.03685605,0.00737121,2.50072e-14
Newton-CG,1b,17,57,69,17,0.03074226,0.00180837,8.71154e-12


# Discusión resultados  
<li> **Número de iteraciones**:  
    Los métodos que tienen ayuda en la selección de la dirección (hessiano) iteran mucho menos, del orden un 10%-50%, precisamente por esa ayuda en la selección de la dirección.</li>
<li> **Número de evaluaciones de la Función y graidente**:  
    Al igual que en las iteraciones, los métodos que tienen ayuda en la selección de la dirección (hessiano) evaluan mucho menos la función y el gradiente.</li>
<li> **Tiempo de computación**:  
    Los métodos que tienen ayuda en la selección de la dirección (hessiano) tardan menos en computar. Esto es debido al menor número de iteraciones y no tanto al esfuerzo computacional ya que cada iteración tarda más, x10.</li>
<li> **Error**:   
    Al igual que antes, los métodos que tienen ayuda en la selección de la dirección (hessiano) tienen menos error ya que cerca de la solución se pueden acercar más ella por dicha ayuda.</li>

### c) Estimación del valor de los coeficientes de la regresión implementando:
<li> i. Método del Gradiente.</li>
<li> ii. Método de Newton.</li>
<li> iii. Método de Quasi-Newton.</li>
<li> iv. Método del Gradiente Estocástico.</li>

### i. (0.5 punto) Método del Gradiente.

Este método solo utiliza la condición de que la derivada primera sobre beta sea menor que cero.  
Produce un número alto de iteraciones.

In [15]:
# Definimos índices
(n,m) = X.shape
beta_ls = np.zeros(m) 
n_iter=1000

# Definimos variables que usaremos para almacenar valores la Función Objetivo, alfa e iteraciones:
OF_iter = np.zeros(n_iter)
tol_iter = np.zeros(n_iter)
alpha_iter = np.zeros(n_iter)

# Inicializamos
i = 0;
tol = 1e4;
epsilon = 1e-3;
time_start = time.clock()
cont_grad = 0
cont_alpha = 0

while (i <= n_iter-1) and (tol>epsilon):
    # Dirección
    cont_grad += 1
    grad = least_sq_reg_jac(beta_ls,X,Y)
    ddirect = -grad
    
    # Paso. Calculamos el paso más apropiado con la regla de Armijo.
    alpha=0.05
    sigma = 0.05
    beta= 0.05
    while (least_sq_reg(beta_ls+alpha*ddirect,X,Y) > least_sq_reg(beta_ls,X,Y)+alpha*sigma*np.dot(grad,ddirect)):
        alpha=alpha*beta
        cont_alpha += 1
    
    # Movimiento
    beta_ls = beta_ls + alpha*ddirect
    OF_iter[i] =least_sq_reg(beta_ls, X, Y)
    tol = np.linalg.norm(least_sq_reg_jac(beta_ls,X,Y), ord=2)
    tol_iter[i] = tol
    alpha_iter[i]=alpha

    # Aumentamos el contador
    i += 1

    
time_elapsed = (time.clock() - time_start) 
error = np.linalg.norm(beta_ls_exact.T - beta_ls,ord=2)/np.linalg.norm(beta_ls_exact.T,ord=2)
print('beta: ',beta_ls)

ejercicio = '1c'
method = 'Gradient'
aux = [ejercicio,i, cont_grad, cont_alpha, '{:.8f}'.format(time_elapsed), '{:.8f}'.format(time_elapsed/i), error]
aux = pd.DataFrame(aux).T
aux.columns = ['ejercicio','n_iter','n_grad','n_alpha','total_time_elapsed','iter_time_elapsed','error']
aux.index = [method]
df_comparativo = pd.concat([df_comparativo,aux])

beta:  [-0.31697509 -0.00662153 -1.12842702 -0.13851037 -1.45076361  1.37383216
  3.40309994  2.94511846 -1.67679779 -0.33605802 -5.14917015  3.57361842
 -2.35532995 -0.34018469 -3.39500485  1.91605871  0.37329151 -1.03390348
  0.32135236 -3.31478051  3.07825667 -2.26069776  1.03719729  3.85672777
 -4.39511457 -0.95608087 -3.62786652  3.85113841 -1.22576828 -2.69745281
  1.68515777  0.3358071   3.24001294  2.56048493  4.6351544  -3.56378585
 -4.1704756   4.22150619  4.5477107  -1.12335419 -0.13198606 -1.38565998
  2.5188029   2.6887277  -2.54306698 -0.87315576 -2.71195244 -4.7478493
 -4.34731996 -3.53495752 -4.51089892 -0.80290652 -4.20756185  3.50862689
  2.05549256  2.69780797  1.22755207  1.5352091  -3.95129921 -4.21900455
  4.52281705 -1.1518718  -4.20916379 -3.38414138  3.6791647  -2.68028956
 -0.28857676  1.64044801  4.62454051 -4.59274963 -0.15933797  4.97730581
 -3.39446043 -3.59152118 -3.88667801  0.11700816 -1.69522458  1.85461978
  0.51507053  0.80711484  0.17746465  4.99012

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



### ii. (0.5 punto) Método de Newton.

Además del Jacobiano (derivada de orden 1) introducimos el hessiano (derivada de orden 2) para iterar menos veces ya que la elección de la dirección es más orientada.  

In [16]:
# Definimos índices
(n,m) = X.shape
beta_ls = np.zeros(m) 
n_iter=1000

# Definimos variables que usaremos para almacenar valores la Función Objetivo, alfa e iteraciones:
OF_iter = np.zeros(n_iter)
tol_iter = np.zeros(n_iter)
alpha_iter = np.zeros(n_iter)

# Inicializamos
i = 0;
tol = 1e4;
epsilon = 1e-3;
time_start = time.clock()
cont_grad = 0
cont_hess = 0
cont_alpha = 0

while (i <= n_iter-1) and (tol>epsilon):
    # Dirección
    cont_grad += 1
    cont_hess += 1
    grad = least_sq_reg_jac(beta_ls,X,Y)
    hess = least_sq_reg_hess(beta_ls,X,Y)
    ddirect = -np.dot(np.linalg.inv(hess),grad)
    
    # Paso. Calculamos el paso más apropiado con la regla de Armijo.
    alpha=0.05
    sigma = 0.05
    beta= 0.05
    while (least_sq_reg(beta_ls+alpha*ddirect,X,Y) > least_sq_reg(beta_ls,X,Y)+alpha*sigma*np.dot(grad,ddirect)):
        alpha=alpha*beta
        cont_alpha += 1
    
    # Movimiento
    beta_ls = beta_ls + alpha*ddirect
    OF_iter[i] =least_sq_reg(beta_ls, X, Y)
    tol = np.linalg.norm(least_sq_reg_jac(beta_ls,X,Y), ord=2)
    tol_iter[i] = tol
    alpha_iter[i] = alpha

    # Aumentamos el contador
    i += 1


time_elapsed = (time.clock() - time_start) 
error = np.linalg.norm(beta_ls_exact.T - beta_ls,ord=2)/np.linalg.norm(beta_ls_exact.T,ord=2)
print('beta: ',beta_ls)

ejercicio = '1c'
method = 'Newton'
aux = [ejercicio,i, cont_grad, cont_alpha, '{:.8f}'.format(time_elapsed), '{:.8f}'.format(time_elapsed/i), error]
aux = pd.DataFrame(aux).T
aux.columns = ['ejercicio','n_iter','n_grad','n_alpha','total_time_elapsed','iter_time_elapsed','error']
aux.index = [method]
df_comparativo = pd.concat([df_comparativo,aux])

beta:  [-0.15233413  0.08546847 -1.17631575 -0.14108304 -1.40110842  1.45466705
  3.44572392  2.91711792 -1.61313215 -0.1533509  -5.21591824  3.83021274
 -2.49119514 -0.50809872 -3.48931598  1.93092299  0.36183478 -1.07269883
  0.43490732 -3.30353151  3.02848063 -2.24520813  1.09712706  3.97565123
 -4.55225201 -1.08153869 -3.62442505  3.86066218 -1.30016656 -2.93455732
  1.80011328  0.38632804  3.26473812  2.59868209  4.79013721 -3.82204703
 -4.20181938  4.29823579  4.69180632 -1.10880056 -0.1686732  -1.47431858
  2.66300585  2.71941878 -2.64873993 -1.11712918 -2.65617811 -4.73107722
 -4.39730113 -3.65452986 -4.53338754 -0.7289705  -4.38038218  3.55034352
  2.25542915  2.80988335  1.26239186  1.60027249 -3.90060293 -4.38904362
  4.69152958 -1.23971773 -4.37217525 -3.43485853  3.64172356 -2.74194167
 -0.24696555  1.67912417  4.68899761 -4.8367725  -0.03785452  5.04008999
 -3.5481505  -3.7134743  -4.10746104  0.10834192 -1.574356    2.02058665
  0.57063039  0.79287304  0.08262846  5.0219

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



### iii. (0.5 punto) Método de Quasi-Newton.

El método Quasi-Newton no utiliza el hessiano lo que agiliza su computación penalizando algo su precisión.  

In [17]:
# Definimos índices
(n,m) = X.shape

beta_ls = np.zeros(m) 
n_iter = 1000

# Definimos variables que usaremos para almacenar valores la Función Objetivo, alfa e iteraciones:
OF_iter = np.zeros(n_iter)
tol_iter = np.zeros(n_iter)

# Inicializamos
i = 0;
tol = 1e3;
epsilon = 1e-3;
alpha = 1
hess = least_sq_reg_hess(beta_ls,X,Y)
time_start = time.clock()
cont_grad = 0

while (i <= n_iter-1) and (tol>epsilon):
    # Dirección
    cont_grad += 1
    grad = least_sq_reg_jac(beta_ls,X,Y)
    B = hess
    ddirect = -np.dot(np.linalg.inv(B),grad)
    
    # Paso. Calculamos el paso más apropiado con la regla de Armijo.
    sigma = 0.05
    beta= 0.05
    while (least_sq_reg(beta_ls+alpha*ddirect,X,Y) > least_sq_reg(beta_ls,X,Y)+alpha*sigma*np.dot(grad,ddirect)):
        alpha=alpha*beta
        cont_alpha += 1
    
    # Movimiento
    beta_ls0 = beta_ls
    beta_ls = beta_ls + alpha*ddirect  
    OF_iter[i] =least_sq_reg(beta_ls, X, Y)
    s = beta_ls - beta_ls0
    y = least_sq_reg_jac(beta_ls,X,Y) - grad
    hess = B + (y-B*s)*(y-B*s).T/(y-B*s).T*s
    tol=np.linalg.norm(least_sq_reg_jac(beta_ls,X,Y), ord=2)
    tol_iter[i]=tol

    # Aumentamos el contador
    i += 1


time_elapsed = (time.clock() - time_start) 
error = np.linalg.norm(beta_ls_exact.T - beta_ls,ord=2)/np.linalg.norm(beta_ls_exact.T,ord=2)
print('beta: ',beta_ls)

ejercicio = '1c'
method = 'Quasi-Newton'
aux = [ejercicio,i, cont_grad, cont_alpha, '{:.8f}'.format(time_elapsed), '{:.8f}'.format(time_elapsed/i), error]
aux = pd.DataFrame(aux).T
aux.columns = ['ejercicio','n_iter','n_grad','n_alpha','total_time_elapsed','iter_time_elapsed','error']
aux.index = [method]
df_comparativo = pd.concat([df_comparativo,aux])


beta:  [-0.15233413  0.08546847 -1.17631577 -0.14108304 -1.40110843  1.45466706
  3.44572396  2.91711795 -1.61313217 -0.1533509  -5.2159183   3.83021278
 -2.49119516 -0.50809872 -3.48931602  1.93092301  0.36183479 -1.07269884
  0.43490732 -3.30353155  3.02848066 -2.24520816  1.09712707  3.97565127
 -4.55225206 -1.0815387  -3.62442509  3.86066222 -1.30016657 -2.93455736
  1.8001133   0.38632804  3.26473816  2.59868212  4.79013726 -3.82204707
 -4.20181943  4.29823584  4.69180638 -1.10880057 -0.1686732  -1.47431859
  2.66300588  2.71941881 -2.64873996 -1.11712919 -2.65617814 -4.73107727
 -4.39730118 -3.6545299  -4.53338759 -0.72897051 -4.38038223  3.55034356
  2.25542917  2.80988338  1.26239187  1.6002725  -3.90060297 -4.38904367
  4.69152963 -1.23971775 -4.3721753  -3.43485857  3.6417236  -2.7419417
 -0.24696555  1.67912419  4.68899766 -4.83677255 -0.03785452  5.04009005
 -3.54815054 -3.71347434 -4.10746108  0.10834192 -1.57435602  2.02058667
  0.57063039  0.79287305  0.08262846  5.02195

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [18]:
df_comparativo

,ejercicio,error,iter_time_elapsed,n_alpha,n_eval,n_grad,n_hess,n_iter,total_time_elapsed
SLSQP,1b,5.50764e-06,0.00753921,NaN,4310,41,None,41,0.30910774
CG,1b,7.33548e-09,0.00064443,NaN,149,149,None,67,0.04317675
Quasi-Newton (BFGS),1b,3.05078e-14,0.00070219,NaN,191,191,None,133,0.09339162
trust-exact,1b,2.50072e-14,0.00737121,NaN,6,6,6,5,0.03685605
Newton-CG,1b,8.71154e-12,0.00180837,NaN,57,69,17,17,0.03074226
Gradient,1c,0.0358966,0.00049580,2816,NaN,1000,NaN,1000,0.49580111
Newton,1c,1.11513e-08,0.00104918,0,NaN,357,NaN,357,0.37455648
Quasi-Newton,1c,1.66258e-14,0.00217222,0,NaN,1,NaN,1,0.00217222


# Discusión  
<li> **Número de iteraciones**:  
    Los métodos que tienen ayuda en la selección de la dirección (hessiano) iteran muchísimo menos, del orden de menos de un 30%, precisamente por esa ayuda en la selección de la dirección.  
Con respecto a resultado del ejercico 1b, no sé porqué Newton tiene más iteraciones en este caso ya que le damos una ayuda extra con Armijo rule.</li>
<li> **Número de evaluaciones de la Función y gradiente**:  
    Al igual que en las iteraciones, los métodos que tienen ayuda en la selección de la dirección (hessiano) evaluan mucho menos la función y el gradiente.</li>
<li> **Tiempo de computación**:  
    Los métodos que tienen ayuda en la selección de la dirección (hessiano) tardan menos en computar. Esto es debido al menor número de iteraciones y no tanto al esfuerzo computacional ya que cada iteración tarda más, x10.</li>
<li> **Error**:   
    Al igual que antes, los métodos que tienen ayuda en la selección de la dirección (hessiano) tienen menos error ya que cerca de la solución se pueden acercar más ella por dicha ayuda.</li>

### d) La técnica *Least Absolute Regression* busca aquellos $\beta = (\beta_{0}, \beta_{1}, \beta_{2}, \dots, \beta_{k})^T$ que minimizen la suma de los valores absolutos de los residuos:

\begin{align*}
\underset{\beta}{\min} & \quad \sum_{i=1}^{n} |y_{i}-\beta^Tx_{i}|\\
\end{align*}

Se plantea una linealización de esta formulación, implementala en Pyomo, y se compara la solución con la obtenida en a).

# Solución
Variables de decisión:
<li>$\beta_{j} \equiv$ Variables</li>
<li>$z \equiv$ Variable auxiliar</li>


Función Objetivo:

\begin{align*}
\underset{\beta_{j}}{\min} & \quad \sum_{i=1}^{n} |Y_{i}-X_{i,j}\beta_{j}|_{1}
\end{align*}
  
Linealización de la Función Objetivo:

\begin{align*}
\underset{\beta,z}{\min} & \quad \sum_{i=1}^{n} z_{i}
\end{align*} 
  
Restricciones:

\begin{align*}
\text{s.t.:} \quad  &\\
\text{(1)} \quad   &-z_{i} \leq \sum_{1}^{n}Y_{i}-X_{i,j}\beta_{j} \leq z_{i}\\
\text{(2)} \quad   &z_{i} \geq 0\\
\end{align*}

In [19]:
np.random.seed(42)
nsample = 1000
nvariable = 100
X0 = np.ones([nsample,1]) # columna Beta_0, luego se concatena con las betas_i
Xi = np.random.random([nsample,nvariable]) # Los valores de x_i están entre 0 y 1
X_real = np.concatenate([X0, Xi],axis=1)
beta_exact = np.random.uniform(-5,5,size=([nvariable+1,1]))
epsilon = np.random.normal(0,1,(nsample,1)) # El error se distribuye según una normal
Y_real = np.dot(X_real,beta_exact) + epsilon


In [20]:
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
opt = SolverFactory("glpk")
from pyomo.opt import SolverFactory

n = 1000;
m = 101;

def LeastAbsoluteRegression(n,m,X,Y):

    model = AbstractModel()
    
    # create indexes
    model.I = RangeSet(1,n) # observations.
    model.J = RangeSet(1,m)  # variables.
    
    # create variables
    model.beta = Var(model.J)  # beta values that we want to get.
    model.z = Var(model.I, domain=NonNegativeReals)  # auxiliar variable to manage to linear the objective function.

    # create Objective Function
    def objective_function(model):
        return sum(model.z[i] for i in model.I)
    model.OF = Objective(rule=objective_function, sense = minimize)

    # create constraints
    # (1.1) linear transformation 1.1 constratints
    def linear_transformation_11_constraint_rule(model,i):
         return -model.z[i] <= Y[i-1][0]-sum(X[i-1,j-1]*model.beta[j] for j in model.J)
    model.linear_transformation_11_constraint = Constraint(model.I, rule = linear_transformation_11_constraint_rule)
    
    # (1.2) linear transformation 1.2 constratints
    def linear_transformation_12_constraint_rule(model,i):
         return Y[i-1][0]-sum(X[i-1,j-1]*model.beta[j] for j in model.J) <= model.z[i]
    model.linear_transformation_12_constraint = Constraint(model.I, rule = linear_transformation_12_constraint_rule)

    instance = model.create_instance()
    instance.dual = Suffix(direction=Suffix.IMPORT)
    results = opt.solve(instance)

    x_sol = np.zeros((m))
    
    for j in range(0,m):
        x_sol[j]=instance.beta[j+1].value
        
    return instance.OF(), x_sol

[OF, x_sol]=LeastAbsoluteRegression(n,m,X_real,Y_real)
[x_sol]

[array([ 0.09052581,  0.19869926, -1.11767149, -0.18293121, -1.35252005,
         1.61902421,  3.33924241,  2.89270415, -1.58115317, -0.00832208,
        -5.16653547,  3.80695085, -2.52756072, -0.60230156, -3.41574713,
         1.94199616,  0.31691582, -1.10140209,  0.34724384, -3.2295255 ,
         3.0541115 , -2.28002653,  1.09166082,  3.89092617, -4.55164708,
        -0.98121112, -3.74559067,  3.81785943, -1.43353035, -3.00302431,
         1.91587336,  0.47522447,  3.20621142,  2.53756488,  4.76148229,
        -3.77023926, -4.29137061,  4.08332937,  4.75149469, -1.07402046,
        -0.1627864 , -1.50364058,  2.75673853,  2.71275847, -2.61001203,
        -1.13673819, -2.64519134, -4.69200552, -4.27041648, -3.60160089,
        -4.46619508, -0.76311173, -4.53933745,  3.61849958,  2.36972625,
         2.75719592,  1.34452077,  1.7098212 , -3.87090893, -4.3579809 ,
         4.6713998 , -1.36776325, -4.36418815, -3.38316912,  3.55917414,
        -2.92798701, -0.26201774,  1.64232862,  4.8